In [238]:
import os
import sys

def cf(filename):
    """
    Preconditions:
      - filename: a string representing the name of the file to be concatenated
      with the directory path.

    Postconditions:
      - Returns a string representing the absolute path of the file after
      concatenation.

    Args:
        filename (str): The name of the file to be concatenated with the
        directory path.

    Returns:
        str: The absolute path of the file after concatenation.
    """
    if 'google.colab' in sys.modules:
        # If running in Colab
        if not os.path.exists('/content/drive'):
            # Mount Google Drive if it's not already mounted
            from google.colab import drive
            drive.mount('/content/drive')
        else:
            print("Google Drive is already mounted.")
        PATH = '/content/drive/MyDrive/'
        data_dir = os.path.join(PATH, filename.lstrip('/'))
    else:
        # If not running in Colab, return the local file path
        data_dir = os.path.join(os.getcwd(), filename)
    return data_dir


##TASK 1

Convert the provided `books_2024.csv`, `bookloans_2024.csv`, and `members_2024.csv` files into JSON format.

In [ ]:
import csv
import json
import datetime

# Constants for headers
BOOK_HEADERS = ['Number', 'Title', 'Author', 'Genre', 'SubGenre', 'Publisher']
MEMBER_HEADERS = ['ID', 'First Name', 'Last Name', 'Gender', 'Email', 'Card Number']

def read_csv_to_dict(filename, expected_headers):
    """ Reads a CSV file and returns a dictionary based on expected headers. """
    try:
        with open(cf(filename), mode='r', encoding='utf-8-sig') as file:
            csv_reader = csv.reader(file)
            headings = next(csv_reader)
            if headings != expected_headers:
                raise ValueError(f"Invalid CSV file: headers must be {expected_headers}")

            data_dict = {}
            for row in csv_reader:
                if len(row) != len(expected_headers):
                    raise ValueError(f"Invalid row: expected {len(expected_headers)} columns but got {len(row)}")
                data_dict[row[0]] = {
                    'Number': row[0],  # Include the book number
                    'Title': row[1],
                    'Author': row[2],
                    'Genre': row[3],
                    'SubGenre': row[4],
                    'Publisher': row[5]
                }
            return data_dict

    except FileNotFoundError:
        raise FileNotFoundError(f"File '{filename}' not found.")
    except Exception as e:
        raise ValueError(f"Error reading CSV file: {e}")

def write_dict_to_json(data_dict, file_path):
    """ Writes a dictionary to a JSON file. """
    try:
        with open(cf(file_path), 'w') as file:
            json.dump(data_dict, file, indent=4)
        print(f"Data saved to file: {file_path}")
    except Exception as e:
        print(f"Error writing to file: {file_path}. {e}")
        return False
    return True

# Read and write book data
library_dictionary = read_csv_to_dict('books_2024.csv', BOOK_HEADERS)
write_dict_to_json(library_dictionary, 'library.json')

# Read and write member data
members_dictionary = read_csv_to_dict('members_2024.csv', MEMBER_HEADERS)
write_dict_to_json(members_dictionary, 'members.json')

def excel_to_date(excel_date: int) -> str:
    """ Converts Excel date to 'YYYY-MM-DD' format. """
    excel_epoch_start = datetime.datetime(1899, 12, 30)
    py_date = excel_epoch_start + datetime.timedelta(days=excel_date)
    return py_date.strftime('%Y-%m-%d')

def convert_csv_no_header_to_json(csv_file, json_file):
    """ Converts a CSV file without headers to a JSON file. """
    headings = ['book_number', 'member_id', 'date_of_loan', 'date_of_return']
    try:
        with open(cf(csv_file), 'r', encoding='utf-8-sig') as file:
            bookloans = csv.DictReader(file, fieldnames=headings)
            rows = list(bookloans)
            for row in rows:
                row['date_of_loan'] = excel_to_date(int(row['date_of_loan']))
                row['date_of_return'] = excel_to_date(int(row['date_of_return']))

        with open(cf(json_file), 'w') as file:
            json.dump(rows, file, indent=4)

    except Exception as e:
        print(f"Error converting CSV to JSON: {e}")

convert_csv_no_header_to_json('bookloans_2024.csv', 'bookloans.json')

# Function to print the contents of a JSON file
def print_json_file(file_path):
    """ Prints the contents of a JSON file. """
    try:
        with open(cf(file_path), 'r') as file:
            data = json.load(file)
            print(f"\nContents of {file_path}:")
            print(json.dumps(data, indent=4))
    except Exception as e:
        print(f"Error reading JSON file: {file_path}. {e}")

# Print the contents of the JSON files created
#print_json_file('library.json')
#print_json_file('members.json')
#print_json_file('bookloans.json')

##TASK 1

Implement the `scan()` method in the `Book` and `Member` classes to simulate the scanning of a book or membership card.
     - Update the JSON file when a member borrows a book.
     - Testing: Use dummy data to test the borrowing operation and include preconditions and postconditions in a docstring.

##CLASS BOOK

In [240]:
class Book:
  def __init__(self, number: str, title: str, author: str, genre: str, subgenre: str, publisher: str):

    """
      Initializes a new Book instance.

      Args:
          number (str): The unique identifier for the book.
          title (str): The title of the book.
          author (str): The author of the book.
          genre (str): The genre of the book.
          subgenre (str): The subgenre of the book.
          publisher (str): The publisher of the book.
    """

    self.number = number
    self.title = title
    self.author = author
    self.genre = genre
    self.subgenre = subgenre
    self.publisher = publisher
    self.available = True
    self.reserved = False

  def reserve(self):
    if self.available:
      self.reserved = True
      self.available = False
      return f"{self.title} has been reserved."
    else:
      return f"{self.title} is currently unavailable for reservation."

  def scan(self):
    return f"Book title: {self.title}\nAuthor: {self.author}\nGenre: {self.genre}\nSub-Genre: {self.subgenre}\nPublisher: {self.publisher}\nAvailable: {self.available}"

  def toggle_availability(self) -> None:
      #Toggles the availability status of the book.
      self.available = not self.available

  def __str__(self):
        """
        Returns a string representation of the book.

        Returns:
            str: A string representation of the book, including its title, author.
        """
        return f"Title: {self.title}\nAuthor: {self.author}\nAvailable: {self.available}\nReserved: {self.reserved}"

##CLASS MEMBER

In [ ]:
def update_library_json(members_dict: dict, file_path: str) -> None:
    """Updates the members JSON file with the latest member data.

    Args:
        members_dict (dict): A dictionary of members with their data.
        file_path (str): The path to the JSON file to update.
    """
    with open(cf(file_path), 'w') as file:
        json.dump(members_dict, file, indent=4)

class Member:
  def __init__(self, id: str, first_name: str, last_name: str, gender: str, email: str, card_number):
    """
    Initialise a Member object.

      Args:
        first_name (str): The first name of the member.
        last_name (str): The last name of the member.
        gender (str): The gender of the member.
        email (str): The email address of the member.
        card_number (str): The initial card number of the member.
    """
    self.ID = ID
    self.id = id
    self.first_name = first_name
    self.last_name = last_name
    self.gender = gender
    self.email = email
    self.card_number = card_number
    self.borrowed_books = []
    self.fines = 0.0
    #extrecting the card count form the card number
    if card_number and card_number != '0': #this means there is no card issued as of yet
      self.card_count = int(card_number.split('-')[1])
    else:
      self.card_count = 0

  def get_member_ID(self):
    return self.ID

  def scan(self) -> str:
    return f"ID: {self.id} Name: {self.first_name} {self.last_name}\nGender: {self.gender}\nEmail address: {self.email}\nCard number: {self.card_number}"

  def __str__(self) -> str:
    books_str = ""
    for book in self.borrowed_books:
      books_str += f"\t{book.title} by {book.author}\t\n"

    return f"Member ID: {self.id}\nName: {self.first_name} {self.last_name}\nEmail: {self.email}\nBorrowed Books: {books_str if books_str else 'None'}."

  def get_borrowed_books(self) -> str:
        '''
        Returns a formatted string containing information about all currently borrowed books in the library.
        The method iterates over the list of borrowed books and concatenates the title and author of each book to a string. The resulting string is then returned.
        Returns:
            str: A formatted string containing the title and author of each currently borrowed book in the library.

        '''
        books_str = "\tBooks borrowed so far:\n"
        for book in self.borrowed_books:
            books_str += f"\t\t{book.title} by {book.author}\n"
        return books_str

  def add_borrowed_book(self, book: Book) -> None:
      """
      Adds a borrowed book to the member's record.

      Args:
          book (Book): The Book object being borrowed.
      """
      self.borrowed_books.append(book)

  def remove_borrowed_book(self, book: Book) -> None:
      """
      Removes a borrowed book from the member's record.

      Args:
          book (Book): The Book object being returned.
      """
      try:
        self.borrowed_books.remove(book)
      except ValueError:
        print(f"Book {book.title} not found in borrowed books.")

  def get_member_id(self):
    """
    Gets the membership number of the member.

    Returns:
      str: The member number of the member (membership number of the member).
    """
    return self.id

  def generate_card_number(self) -> str:
    """
    Generates a card number for the member.

    Returns:
      str: The generated card number in the format "XXX-YY", where XXX = the initials and YY = card count.
    """
    member_id = self.get_member_id()
    card_number = f"{self.id}-{self.card_count}"
    return card_number

  def issue_membership_card(self):
    """
    Issues a new membership card for the member.

    Returns:
      str: The updated card number of the member after issuing the new card.
    """
    card_count = self.card_count + 1
    if card_count > 99:
      card_count = 1
    self.card_count = card_count
    self.card_number = self.generate_card_number()
    return self.card_number

  def return_borrowed_book(self, book: Book, members_dict: dict, file_path: str) -> None:
    """
     Allows a member to return a borrowed book and updates the record and the JSON file.
      Preconditions:
        - The book must be in the member's borrowed_books list.
        - The members_dict must contain the member's ID.
      Postconditions:
        - The book is removed from the member's borrowed_books list.
        - The members_dict is updated to reflect the change.
        - The updated members_dict is written to the specified JSON file.
      Args:
        book (Book): The Book object being returned.
        members_dict (dict): The dictionary of members to update.
        file_path (str): The path to the JSON file to update.
    """
    if book in self.borrowed_books:
        self.borrowed_books.remove(book)
        print(f"{book.title} has been returned by {self.first_name}.")
    else:
        print(f"{self.first_name} did not borrow {book.title}.")

    # Update the JSON file
    members_dict[self.id] = {
        'ID': self.id,
        'First Name': self.first_name,
        'Last Name': self.last_name,
        'Gender': self.gender,
        'Email': self.email,
        'Card Number': self.card_number,
        'Borrowed Books': [{'Title': b.title, 'Author': b.author} for b in self.borrowed_books],
    }
    update_library_json(members_dict, file_path)

  def reserve_book(self, book: Book, reserved_dict: dict, file_path: str) -> None:
    """
    Allows a member to reserve a book and updates the record and the JSON file.
    """
    notification = book.reserve()
    if "reserved" in notification:
      # update reserved books dictionary
      reserved_dict[book.number] = {
          'Title': book.title,
          'Reserved by': self.first_name + " " + self.last_name,
          'Member ID': self.id
      }
      # Write this to reserved.json
      with open(cf(file_path), 'w') as file:
        json.dump(reserved_dict, file, indent=4)

    #print(notification)


# Example Usage with Dummy Data
"""if __name__ == "__main__":
    # Create a dummy member
    member1 = Member("1", "Jeff", "Bezzos", "Male", "j.bezz@example.com", "001-1")
    # Create dummy books
    book1 = Book("1", "The Great Gatsby", "F. Scott Fitzgerald", "Fiction", "Classic", "Scribner")
    book2 = Book("2", "1984", "George Orwell", "Dystopian", "Classic", "Secker & Warburg")
    book3 = Book("3", "To Kill a Mockingbird", "Harper Lee", "Fiction", "Classic", "J.B. Lippincott & Co.")

    # Dummy reserved books dictionary
    reserved_dict = {}

    # Test reserving a book
    member1.reserve_book(book1, reserved_dict, 'reserved.json')
    print("\nCurrent Reserved Books:")
    print(json.dumps(reserved_dict, indent=4))

    # Attempt to reserve the same book again
    member1.reserve_book(book1, reserved_dict, 'reserved.json')

    # Reserve another available book
    member1.reserve_book(book2, reserved_dict, 'reserved.json')
    print("\nCurrent Reserved Books:")
    print(json.dumps(reserved_dict, indent=4))

    # Attempt to reserve a book that is already reserved
    member1.reserve_book(book2, reserved_dict, 'reserved.json')

    # Add the book to borrowed books
    member1.add_borrowed_book(book1)

    # Dummy members dictionary
    members_dict = {
        "1": {
            'ID': member1.id,
            'First Name': member1.first_name,
            'Last Name': member1.last_name,
            'Gender': member1.gender,
            'Email': member1.email,
            'Card Number': member1.card_number,
            'Borrowed Books': [{'Title': book1.title, 'Author': book1.author}]
        }
    }

    # Simulate returning the book
    member1.return_borrowed_book(book1, members_dict, 'members.json')

    # Print the updated members dictionary for verification
    print(json.dumps(members_dict, indent=4))
    """

##TESTING CLASSES

###Testing the BOOK CLASS

In [ ]:
def test_book_class():
    print("Testing Book Class")

    # Create a Book instance
    book1 = Book("1", "The Great Gatsby", "F. Scott Fitzgerald", "Fiction", "Classic", "Scribner")

    # Check initial attributes
    assert book1.title == "The Great Gatsby"
    assert book1.author == "F. Scott Fitzgerald"
    assert book1.available is True

    # Test scan method
    print(book1.scan())

    # Toggle availability and check
    book1.toggle_availability()
    assert book1.available is False
    print(book1.scan())

    print("Book Class tests passed!\n")

test_book_class()


###Testing the MEMBER CLASS

In [ ]:
def test_member_class():
    print("Testing Member Class")

    # Create a Member instance
    member1 = Member("001", "John", "Doe", "Male", "john.doe@example.com", "001-0")

    # Check initial attributes
    assert member1.first_name == "John"
    assert member1.card_count == 0

    # Issue a membership card and check the card number
    member1.issue_membership_card()
    assert member1.card_number == "001-1"

    # Add a book to borrowed books
    book1 = Book("1", "The Great Gatsby", "F. Scott Fitzgerald", "Fiction", "Classic", "Scribner")
    member1.add_borrowed_book(book1)

    # Check borrowed books
    assert len(member1.borrowed_books) == 1
    print(member1)

    # Remove the borrowed book
    member1.remove_borrowed_book(book1)
    assert len(member1.borrowed_books) == 0

    print("Member Class tests passed!\n")

test_member_class()


##TASK 3

##Membership system

In [ ]:
class Membership_System:

  def __init__(self):
     self.members_dict = {}

  def apply_for_membership(self, ID, first_name, last_name, gender, email, card_number):
    if ID in self.members_dict:
      raise ValueError("Member ID already exists.")
    member = Member(ID, first_name, last_name, gender, email, card_number)
    self.members_dict[ID] = member

  def get_member_card_count(self, ID):
    if ID not in self.members_dict:
      raise ValueError("Member ID does not exist.")
    return self.members_dict[ID].card_count

def read_members_from_csv(csv_file_path, membership_system):
  with open(cf(csv_file_path), 'r', newline='', encoding='utf-8-sig') as file:
    reader = csv.DictReader(file)
    for row in reader:
      ID = int(row['ID'])
      first_name = row['First Name']
      last_name = row['Last Name']
      gender = row['Gender']
      email = row['Email']
      card_number = row['Card Number']
      membership_system.apply_for_membership(ID, first_name, last_name, gender, email, card_number)

membership_system = Membership_System()

csv_file_path = "members_2024.csv"
read_members_from_csv(csv_file_path, membership_system)

#issuing of membership cards

member_card_issued = list(membership_system.members_dict.keys())

for ID in membership_system.members_dict:
  member = membership_system.members_dict[ID]

  print(f"{ID}'s card number {membership_system.get_member_card_count(ID)} is being updated.")

  card_number_member = member.issue_membership_card()
  print(f"Notification  of new card for Member {ID}: {card_number_member}\n")

#Put the updated members_dict in a JSON file
with open(cf("Membership_Cards.json"), 'w') as json_file:
  members_data = {}
  for ID, member in membership_system.members_dict.items():
    members_data[ID] = {
        'First Name': member.first_name,
        'Last Name': member.last_name,
        'Card Number': member.card_number,
    }
  json.dump(members_data, json_file, indent=4)

##CLASS SMPT SERVER

In [245]:
class FakeSMTPServer:
  from typing import List

  def __init__(self):
        self.sent_emails = []

  def send_mail(self, from_addr: str, to_addrs: List[str], msg: str):
    to_addrs_str = ', '.join(to_addrs) # Add ability to join multiple recipients for better readablity
    self.sent_emails.append({'from': from_addr, 'to': to_addrs, 'msg': msg})

  def print_sent_emails(self):
      for email in self.sent_emails:
          print("From: {}".format(email['from']))
          print("To: {}".format(email['to']))
          print("Message: {}".format(email['msg']))
          print()

  def quit(self):
    print("Fake SMTP server is shutting down.")

def send_notification_email(member: Member, message: str, fake_smtp_server: FakeSMTPServer):
    """
    Sends a notification email to a member.

    Args:
        member (Member): The Member object to whom the email will be sent.
        message (str): The message to include in the email.
        smtp_server (SMTP): An SMTP server object to use for sending the email.
    """
    #use first and last name for the email
    full_name = f"{member.first_name} {member.last_name}"
    # Set up the email content
    subject = "Library notification"
    body = f"Dear {full_name},\n\n{message}\n\nSincerely,\nThe Library Team"
    from_address = "library@example.com"
    to_address = [member.email]

    # Send the email using the fake SMTP server
    fake_smtp_server.send_email(from_address, to_address, body)

##CLASS LIBRARY

In [246]:
from datetime import datetime

class Library:
  """
  A class representing a library.

  Attributes:
    name (str): The name of the Library.
    books (dict): A Dictionary of Book objects keyed by their Book Numbers.
    members (dict): a dictionary of Memebr objects keyed by their ID numbers.

  Methods:
    __init__(self, name: str)
    get_member(self, member_id: int) --> Member
    on_shelf(self) --> str
    add_book(self, book: Book) -> none
    add_member(self, member: Member) -> none
    borrow_book(self, ID: str, Number: str) -> bool
    return_book(self, ID: str, Number: str) -> None
    collect_payment(self, member: Member, amount: Float) -> None
  """
  def __init__(self, name: str, books_data: dict = None):
    """
    Initialises a new instance of the Library class.

    Args: name(str): The name of the Library.
    """
    self.name = name
    self.books = {}
    self.members = {}
    self.books_data = books_data

  def __str__(self):
    books_str = "\n\t".join(f"{number}: {book.title} by {book.author}" for number, book in self.books.items())

    available_books_str = "\n\t".join(f"{number}: {book.title} by {book.author}" for number in self.books if self.book[number].available)

    members_str = "\n\t".join(f"{ID}: {member.first_name} {member.last_name}" for ID, member in self.members.items())

    return f"\nLibrary('{self.name}')\nBooks:\n\t{books_str}\nAvailable books:\n\t{available_books_str}\nMembers:\n\t{members_str}"

  def book_scanner(self, Number):
    if self.books_data is not None and isinstance(self.books_data, dict) and "data" in self.books_data and self.books_data["data"] is not None: # Check if books_data is a dictionary and contains the 'data' key
      scanned_book = self.books_data["data"].get(Number)
      if scanned_book is not None:
        return scanned_book.scan()
      else:
        return "Book not found."
    else:
      return "Error loading book data."


  def get_member(self, ID):
    """
    Returns the member object corresponding to the given member ID.

    Args:
      member_id (str): the ID of the member to retrieve.

      Returns:
        Member: The member object corresponding to the given ID, or none if the member ID is invalid.
    """
    return self.members.get(ID)

  def on_shelf(self): #function to help with feedback
    available_books_str = ""
    ct = 0
    for number in self.books.items():
      if book.available:
        ct+=1
        available_books_str += f"{number}: {book.title} by {book.author}\n\t"
    return f"\n{ct} of {len(self.books.items())} books left on the shelf:\n\t{available_books_str}\n"

  def add_book(self, book: Book) -> None:
    """
    Adds a book to the library's collection.

    Args:
      book (Book): the book object to add.
    """
    self.books[book.number] = book

  def add_member(self, member: Member) -> None:
    """
    Adds a member to the library.

    Args:
      member (Member): the member object to add.
    """
    self.members[member.id] = member

  def borrow_book(self, ID: str, number: str) -> bool:
    """
    Allows a member to borrow a book from the library.

    Args:
      member_id (str): the ID of the member borrowing the book.
      book_number (str): the number of the book being borrowed.

    Returns:
      bool: true if the book was successfully borrowed, false otherwise.
    """
    if ID not in self.members: #check if the member exists in the library records
      raise ValueError("Member ID does not exist.")
    if number not in self.books: #check if the book exists in the library records
      raise ValueError("Book number does not exist.")
    if not self.books[number].available: #check if the book is currently available
      raise ValueError("Book is currently unavailable.")
    self.members[ID].add_borrowed_book(self.books[number]) #update book availability status and members borrowing record
    self.books[number].available = False

  def return_book(self, ID: str, number: str, fake_smtp_server: FakeSMTPServer) -> bool:
    """
      Allows a member to return a book to the library.
      Charges a late fee.
      Collects money owed.

      Args:
          member_id (str): The ID number of the member returning the book.
          isbn (str): The ISBN number of the book being returned.
      Returns:
          bool: True if the book was successfully returned, False otherwise.
    """
    if ID not in self.members: #check if the member exists in the library records
      raise ValueError("Member ID does not exist.")

    if number not in self.books: #check if the book exists in the library records
      raise ValueError("Book number does not exist.")

    if self.books[number] not in self.members[ID].borrowed_books: #check if the book was borrowed by the member
      raise ValueError("Book was not borrowed by the member. Consider a reservation")
    # Update book availability status, member's borrowing record, and collect any outstanding fines
    self.books[number].available = True
    self.members[ID].remove_borrowed_book(self.books[number])

    # simulate possible late return and fee, if so for a late book return
    # assume fee of £0.50 per day late
    with open(cf('bookloans.json'), 'r') as f:
      loan_data = json.load(f)

    fine_amount = 1.0
    allowed_loan_period = 14

    for data in loan_data:
      member_ID = data["member_id"]
      book_number = data["book_number"]
      loan_date = datetime.strptime(data["date_of_loan"], "%Y-%m-%d")
      return_date = datetime.strptime(data["date_of_return"], "%Y-%m-%d")

      days_loaned = (return_date - loan_date).days
      if days_loaned > allowed_loan_period:
        late_fee = fine_amount * (days_loaned - allowed_loan_period)
      else:
        late_fee = 0.0

    self.collect_payment(self.members[ID], self.books[number], late_fee, fake_smtp_server)

    return True

  def collect_payment(self, member: Member, book: Book, late_fee, fake_smtp_server: FakeSMTPServer) -> None:
    """
    Makes a late return fee.
    Collects any outstanding fines from a member who is returning a book.
    Args:
        member (Member): The Member object returning the book.
        book (Book): The Book object being returned.
        late_fee (float): computed fee
    """
    member.fines += late_fee
    message = ""
    if member.fines > 0.0:
      # Collect payment and update member's fine record
      max_payment = member.fines
      payment = round(random.uniform(0, max_payment), 2)
      output_str = ""
      if late_fee > 0:
        output_str = f"{member.first_name}, you have been charged with a late return: ${str(late_fee):.2f}. "
      else:
        output_str = f"Thank you for returning the book on time, {member.name}."

      output_str+=f"{member.name}, you have ${member.fines:.2f}in fines. You are required to pay at least ${payment:.2f}."
      print(output_str)
      member.fines -= payment
      message = output_str

    # Send a fake email notification
    send_notification_email(member, message, fake_smtp_server)
    # update book availablity status and member's borrowing record
    self.books[book.number].available = True

##FUNCTIONS

In [ ]:
import random
import json

class FakeSMTPServer:
    def __init__(self):
        self.sent_emails = []

    def send_email(self, to, subject, body):
        self.sent_emails.append((to, subject, body))

    def print_sent_emails(self):
        for to, subject, body in self.sent_emails:
            print(f"To: {to}\nSubject: {subject}\nBody:\n{body}\n{'-'*40}")

def populate_library_books(library, books_dict):
    for book_id, book_data in books_dict.items():
        book = Book(
            number=book_data.get('Number'),
            title=book_data.get('Title'),
            author=book_data.get('Author'),
            genre=book_data.get('Genre'),
            subgenre=book_data.get('SubGenre'),
            publisher=book_data.get('Publisher')
        )
        library.add_book(book)

def populate_members(library, members_dict):
    for member_id, member_data in members_dict.items():
        member = Member(
            id=member_data.get('ID'),
            first_name=member_data.get('First Name'),
            last_name=member_data.get('Last Name'),
            gender=member_data.get('Gender'),
            email=member_data.get('Email'),
            card_number=member_data.get('Card Number')
        )
        library.add_member(member)

def borrow_random_book(library):
    member_ids = list(library.members.keys())
    book_numbers = list(library.books.keys())
    member_id = random.choice(member_ids)
    book_number = random.choice(book_numbers)
    book = library.books.get(book_number)

    if book and book.available:
        result = library.borrow_book(member_id, book_number)
        if result:
            print(f"\nMember #{member_id} has borrowed book #{book_number}")
            m = library.members.get(member_id)
            print(m.get_borrowed_books())
    else:
        print(f"*** Library Message: Book #{book_number} is not available to borrow. ***")

def return_random_book(library, fake_smtp_server):
    borrowed_books = [(member_id, book) for member_id, member in library.members.items() for book in member.borrowed_books]

    if not borrowed_books:
        print("No borrowed books found.")
        return

    member_id, book = random.choice(borrowed_books)
    result = library.return_book(member_id, book.number, fake_smtp_server)
    if result:
        print(f"\nMember #{member_id} has returned the book #{book.number}")

def main():
    fake_smtp_server = FakeSMTPServer()

    # Load library and member data from JSON
    with open(cf('library.json')) as f:
        books_dict = json.load(f)

    with open(cf('members.json')) as f:
        members_dict = json.load(f)

    print("A simulation to demonstrate the transaction nature of a lending library.\n")
    print("+-+-+-+-+-+-+-+ +-+-+-+-+-+-+-+ +-+-+-+-+-+-+-+-+-+-+ +-+-+-+-+-+-+ ")
    print("|L|e|n|d|i|n|g| |L|i|b|r|a|r|y| |M|a|n|a|g|e|m|e|n|t| |S|y|s|t|e|m|")
    print("+-+-+-+-+-+-+-+ +-+-+-+-+-+-+-+ +-+-+-+-+-+-+-+-+-+-+ +-+-+-+-+-+-+ ")

    library = Library("Rebelky Elite")  # Placeholder for actual library object
    populate_library_books(library, books_dict)
    populate_members(library, members_dict)

    # Simulate random borrows
    for _ in range(10):
        borrow_random_book(library)

    # Simulate returns
    for _ in range(10):
        return_random_book(library, fake_smtp_server)

    # Check sent emails
    fake_smtp_server.print_sent_emails()

if __name__ == "__main__":
    main()


#Task 5

###Question 1:

I feel the current notification system is adequate for sending messages with regards to library interactions, such as borrowing and returning books. However, with my admitidly limited knowledge of python I do believe that the notification system could be made better. We would want to improve its flexibility and extensibility for future demands on the system by the library. There are several enhancements I would consider:

###Different Notification pathways:

- Multiple pathway support: for example we could incorporate different types of notifications dependant on the urgency of the notification. Eg, emails, SMS's, push notifications, etc.  we could also get user preferences.

- User preference: as mentioned above we could customise notifications dependant on the users preferences.

